In [1]:
import os

In [2]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [3]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import glob
class ImageFolderWithPaths(datasets.ImageFolder):
 # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

PATH = "three_label_classifier.pt"


In [6]:
# Training 25672 Total - 13000 Healthy / 11000 Has_DKD / 1,700 CKD_noDM
# Validation 18071 Total - 15,988 Healthy / 1879 Has_DKD / 204 CKD_noDM
# Testing 14861 Total - 11,470 Healthy / 2318 Has_DKD / 1,073 CKD_noDM

# Try to find an equal number of patches from each set do better sampling

plt.ion()   # interactive mode
data_transforms = transforms.Compose([
        transforms.RandomResizedCrop(512),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

temp_DKD = glob.glob('/data1/DKDimages/Three_Label_WSI/KPMP_WSI/Has_DKD/*')
wsi_HasDKD = []
wsi_all = []
for x in temp_DKD:
    
    dkd_split = x.split("/")[-1].split(".")[0].split('_')[0]
#     print(dkd_split)
    wsi_HasDKD.append(dkd_split)
    wsi_all.append(dkd_split)
# print("has DKD",wsi_HasDKD)
# print(len(wsi_HasDKD))
temp_Healthy = glob.glob('/data1/DKDimages/Three_Label_WSI/KPMP_WSI/Healthy/*')
wsi_Healthy = []
for x in temp_Healthy:
    
    dkd_split = x.split("/")[-1].split(".")[0].split('_')[0]
#     print(dkd_split)
    wsi_Healthy.append(dkd_split)
    wsi_all.append(dkd_split)
# print("Healthy",wsi_Healthy)
# print(len(wsi_Healthy))
# print(wsi_all)
# print("all len",len(wsi_all))

temp_CKD_noDM = glob.glob('/data1/DKDimages/Three_Label_WSI/KPMP_WSI/CKD_noDM/*')
wsi_CKD_noDM = []
for x in temp_CKD_noDM:
    
    dkd_split = x.split("/")[-1].split(".")[0].split('_')[0]
#     print(dkd_split)
    wsi_CKD_noDM.append(dkd_split)
    wsi_all.append(dkd_split)
# print("ckd",wsi_CKD_noDM)
# print(len(wsi_CKD_noDM))


# print(wsi_all)
# print("all len",len(wsi_all))
wsi_dict = dict()
for i in wsi_all:
    wsi_dict[i] = dict()
    wsi_dict[i]["Healthy"] = 0
    wsi_dict[i]["Has_DKD"] = 0
    wsi_dict[i]["CKD_noDM"] = 0
# print(wsi_dict)


params = {'batch_size': 4,
          'shuffle': True,
          'num_workers': 8}

class_names = ('CKD_noDM','Has_DKD', 'Healthy', )
# data_dir = '/projectnb/vkolagrp/Tsurendr/Deepslide-NewLabels/train_folder/'
# print(os.path.exists(data_dir))

# training_set = Dataset((os.path.join(data_dir, 'train')), labels)
# training_generator = torch.utils.data.DataLoader(training_set, **params)

# validation_set = Dataset(os.path.join(data_dir, 'val'), labels)
# validation_generator = torch.utils.data.DataLoader(validation_set, **params)



# image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
#                                           data_transforms[x])
#                   for x in ['train', 'val']}


from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
for train_index, test_index in kf.split('/data1/DKDimages/All_patch_Folders/*/*'):
    print(train_index,test_index)

# image_datasets_train = ImageFolderWithPaths('/data1/DKDimages/Pyhist_Patches/train', transform = data_transforms)
# print(len(image_datasets_train))
# image_datasets_val = ImageFolderWithPaths('/data1/DKDimages/Pyhist_Patches/val', transform = data_transforms)
# print(len(image_datasets_val))
# image_datasets_test = ImageFolderWithPaths('/data1/DKDimages/Pyhist_Patches/test', transform = data_transforms)
# print(len(image_datasets_test))
# # print(image_datasets_val)
# print("validation set length: ", len(image_datasets_val))
# dataloader_train = torch.utils.data.DataLoader(image_datasets_train, batch_size=4,
#                                              shuffle=True, num_workers=8)
# dataloader_val = torch.utils.data.DataLoader(image_datasets_val, batch_size=4,
#                                              shuffle=True, num_workers=8)
# dataloader_test = torch.utils.data.DataLoader(image_datasets_test, batch_size=4,
#                                              shuffle=False, num_workers=8)
# # dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
# #                                              shuffle=True, num_workers=8)
# #                for x in ['train', 'val']}
# print("Dataloader size: ", len(dataloader_val))
# dataset_sizes_train = len(image_datasets_train)
# dataset_sizes_val = len(image_datasets_val)
# # class_names = image_datasets['train'].classes

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# path_sum = 0
# # for i, data in enumerate(dataloader_val):
# #     images,labels,paths = data
# # #     print( paths[3])
# #     if paths[0]: 
# #         path_sum+=1
# #         print("path 0", paths[0])
# #     if paths[1]:
# #         path_sum+=1
# #         print("path 1", paths[1])
        
# #     try:
# #         if paths[2]:
# #             path_sum+=1
# #             print("path 2", paths[2])
            
# #     except IndexError:
# #         pass
    
# #     try:
# #         if paths[3]:
# #             path_sum+=1
# #             print("path 3", paths[3])
            
# #     except IndexError:
# #         pass
# # print("done")
# # print(path_sum)
# def imshow(inp, title=None):
#     """Imshow for Tensor."""
#     inp = inp.numpy().transpose((1, 2, 0))
#     mean = np.array([0.485, 0.456, 0.406])
#     std = np.array([0.229, 0.224, 0.225])
#     inp = std * inp + mean
#     inp = np.clip(inp, 0, 1)
#     plt.imshow(inp)
#     if title is not None:
#         plt.title(title)
#     plt.pause(0.001)  # pause a bit so that plots are updated


# # # Get a batch of training data
# inputs, classes, paths = next(iter(dataloader_train))
# # print(classes)
# # print(inputs)
# # Make a grid from batch
# out = torchvision.utils.make_grid(inputs)

# imshow(out, title=[class_names[x] for x in classes])



[19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18] [19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37]


In [ ]:

def train_model(model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()
#     print("hello")
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            if phase == "train":
                
                for inputs, labels, paths in dataloader_train:
                    inputs = inputs.to(device)
                    labels = labels.to(device)
#                     print(min(labels))
#                     print(max(labels))
#                     print(inputs.shape)
        
                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
#                         print(torch.max(outputs, 1))
#                         print(torch.min(outputs, 1))
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
            
            
            if phase == "val":
                
                for inputs, labels, paths in dataloader_val:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
            
            if phase == 'train':
                scheduler.step()
            if phase == 'train':
                epoch_loss = running_loss / dataset_sizes_train
                epoch_acc = running_corrects.double() / dataset_sizes_train
            if phase == 'val':
                epoch_loss = running_loss / dataset_sizes_val
                epoch_acc = running_corrects.double() / dataset_sizes_val
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    torch.save(model, PATH)
    return model



#Whole Covnet        
# model_ft = models.resnet18(pretrained=True)
# num_ftrs = model_ft.fc.in_features
# # Here the size of each output sample is set to 2.
# # Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
# model_ft.fc = nn.Linear(num_ftrs, 2)

# model_ft = model_ft.to(device)

# criterion = nn.CrossEntropyLoss()

# # Observe that all parameters are being optimized
# optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# # Decay LR by a factor of 0.1 every 7 epochs
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
#                        num_epochs=25)
# visualize_model(model_ft)

#final layer ########################

model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
model_conv.outlayer = nn.Linear(512 * 1 * 1, 3)
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, len(class_names))

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)


model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=20)



In [ ]:
import seaborn as sns
def visualize_model(model, num_images,dataloader):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()
    path_sum = 0
    len_class_names = len(class_names)
    confusion_matrix = torch.zeros(len_class_names, len_class_names)
    with torch.no_grad():
        for i, data in enumerate(dataloader):
            inputs, labels, filenames = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            print(labels) 

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
        
#             print(filenames)
            try:
                if filenames[0]: 
                    path_sum+=1
#                     print("path 0", filenames[0])
#                     print("prediction: ", class_names[preds[0]])
                    temp = filenames[0].split("/")[-1].split(".")[0].split('_')[0]
    #                 dict_key = temp[0]+"_"+temp[1]+"_"+temp[2]+"_"+temp[3]
                    dict_key = temp
    #                 print(dict_key)
                    if dict_key in wsi_all:
                        if class_names[preds[0]] == "Has_DKD":
                            wsi_dict[dict_key]["Has_DKD"] += 1
                        elif class_names[preds[0]] == "Healthy":
                            wsi_dict[dict_key]["Healthy"] += 1
                        elif class_names[preds[0]] == "CKD_noDM":
                            wsi_dict[dict_key]["CKD_noDM"] += 1
            except IndexError:
                pass
            try:
                
                if filenames[1]:
                    path_sum+=1
#                     print("path 1", filenames[1])
#                     print("prediction: ", class_names[preds[1]])
                    temp = filenames[1].split("/")[-1].split(".")[0].split('_')[0]
    #                 dict_key = temp[0]+"_"+temp[1]+"_"+temp[2]+"_"+temp[3]
                    dict_key = temp
    #                 print(dict_key)                
                    if dict_key in wsi_all:
                        if class_names[preds[1]] == "Has_DKD":
                            wsi_dict[dict_key]["Has_DKD"] += 1
                        elif class_names[preds[1]] == "Healthy":
                            wsi_dict[dict_key]["Healthy"] += 1
                        elif class_names[preds[1]] == "CKD_noDM":
                            wsi_dict[dict_key]["CKD_noDM"] += 1
    #             print(filenames[2])
            except IndexError:
                pass
            try:
                if filenames[2]:
                    path_sum+=1
#                     print("path 2", filenames[2])
#                     print("prediction: ", class_names[preds[2]])
                    temp = filenames[2].split("/")[-1].split(".")[0].split('_')[0]
                    dict_key = temp
#                     dict_key = temp[0]+"_"+temp[1]+"_"+temp[2]+"_"+temp[3]
#                     print(dict_key)                    
                    if dict_key in wsi_all:
                        if class_names[preds[2]] == "Has_DKD":
                            wsi_dict[dict_key]["Has_DKD"] += 1
                        elif class_names[preds[2]] == "Healthy":
                            wsi_dict[dict_key]["Healthy"] += 1
                        elif class_names[preds[2]] == "CKD_noDM":
                            wsi_dict[dict_key]["CKD_noDM"] += 1
            except IndexError:
                pass

            try:
                if filenames[3]:
                    path_sum+=1
#                     print("path 3", filenames[3])
#                     print("prediction: ", class_names[preds[3]])
                    temp = filenames[3].split("/")[-1].split(".")[0].split('_')[0]
                    dict_key = temp
#                     dict_key = temp[0]+"_"+temp[1]+"_"+temp[2]+"_"+temp[3]
#                     print(dict_key)
                    if dict_key in wsi_all:
                        if class_names[preds[3]] == "Has_DKD":
                            wsi_dict[dict_key]["Has_DKD"] += 1
                        elif class_names[preds[3]] == "Healthy":
                            wsi_dict[dict_key]["Healthy"] += 1
                        elif class_names[preds[3]] == "CKD_noDM":
                            wsi_dict[dict_key]["CKD_noDM"] += 1
            except IndexError:
                pass
        
            for t, p in zip(labels.view(-1), preds.view(-1)):
                            confusion_matrix[t.long(), p.long()] += 1

#             print(filenames[0])
#             [print(class_names[x]) for x in preds]
        
#             for j in range(inputs.size()[0]):
#                 images_so_far += 1
#                 ax = plt.subplot(num_images//2, 2, images_so_far)
#                 ax.axis('off')
#                 ax.set_title('predicted: {}'.format(class_names[preds[j]]))
#                 imshow(inputs.cpu().data[j])

#                 if images_so_far == num_images:
#                     model.train(mode=was_training)
#                     return
#         print(confusion_matrix)
#         print(confusion_matrix.diag()/confusion_matrix.sum(1))

#         sns.heatmap(confusion_matrix, annot=True)


        ax= plt.subplot()
        sns.heatmap(confusion_matrix, annot=True, fmt='g', ax=ax);  #annot=True to annotate cells, ftm='g' to disable scientific notation
        
        # labels, title and ticks
        ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
        ax.set_title('Confusion Matrix'); 
        ax.xaxis.set_ticklabels(['CKD_noDM', 'Has_DKD','Healthy']); ax.yaxis.set_ticklabels(['CKD_noDM', 'Has_DKD','Healthy']);
        print('Predicted label accuracy',(confusion_matrix.diag()/confusion_matrix.sum(1)))
        model.train(mode=was_training)

model = torch.load(PATH)
model.eval()
visualize_model(model,6,dataloader_val)
print("Validation Slide Status: ")
# print(wsi_all)
for k in wsi_all:
#     print(k)
#     print("WSI: ", k, " healthy: ", wsi_dict[k]['Healthy'], "has DKD: ", wsi_dict[k]['Has_DKD'],"CKD_noDM" ,wsi_dict[k]['CKD_noDM'])
    if (max(wsi_dict[k]['Healthy'], wsi_dict[k]['Has_DKD'],wsi_dict[k]['CKD_noDM']) == wsi_dict[k]['Healthy']) and (wsi_dict[k]['Healthy'] != 0 or wsi_dict[k]['Has_DKD'] != 0 or wsi_dict[k]['CKD_noDM'] != 0 ):
        print("WSI: ", k, "Status: Healthy" )
    elif max(wsi_dict[k]['Healthy'], wsi_dict[k]['Has_DKD'],wsi_dict[k]['CKD_noDM']) == wsi_dict[k]['Has_DKD'] and (wsi_dict[k]['Healthy'] != 0 or wsi_dict[k]['Has_DKD'] != 0 or wsi_dict[k]['CKD_noDM'] != 0 ):
        print("WSI: ", k, "Status: Has_DKD" )
    elif max(wsi_dict[k]['Healthy'], wsi_dict[k]['Has_DKD'],wsi_dict[k]['CKD_noDM']) == wsi_dict[k]['CKD_noDM'] and (wsi_dict[k]['Healthy'] != 0 or wsi_dict[k]['Has_DKD'] != 0 or wsi_dict[k]['CKD_noDM'] != 0 ):
        print("WSI: ", k, "Status: CKD_noDM" )
plt.ioff()
plt.show()        

In [ ]:
# dataiter = iter(dataloader_test)
# images, labels, paths = dataiter.next()
# images = images.cuda()
# labels = labels.cuda()
# for k in range(4): 
#     imshow(images.cpu().data[k]) 
# print('GroundTruth: ', ' '.join('%5s' % class_names[labels[j]] for j in range(4)))

# outputs = model_conv(images)

# _, predicted = torch.max(outputs, 1)

# print('Predicted: ', ' '.join('%5s' % class_names[predicted[j]]
#                               for j in range(4)))
# correct = 0
# total = 0
# # since we're not training, we don't need to calculate the gradients for our outputs
# with torch.no_grad():
#     path_sum = 0
#     for i, data in enumerate(dataloader_test):
#         inputs, labels, filenames = data
#         images = images.cuda() 
#         labels = labels.cuda()
   
#         # calculate outputs by running images through the network
#         outputs = model_conv(images)
#         # the class with the highest energy is what we choose as prediction
#         _, preds = torch.max(outputs.data, 1)
#         total += labels.size(0)

        
#         try:
#             if filenames[0]:
#                 path_sum+=1
#                 print("path 1", filenames[0])
#                 print("prediction: ", class_names[preds[0]])
#                 temp = filenames[1].split("/")[-1].split(".")[0].split('_')[0]
#     #                 dict_key = temp[0]+"_"+temp[1]+"_"+temp[2]+"_"+temp[3]
#                 dict_key = temp
#     #             print(dict_key)                
#                 if dict_key in wsi_all:
#                     if class_names[preds[0]] == "Has_DKD":
#                         wsi_dict[dict_key]["Has_DKD"] += 1
#                     elif class_names[preds[0]] == "Healthy":
#                         wsi_dict[dict_key]["Healthy"] += 1
#                     elif class_names[preds[0]] == "CKD_noDM":
#                         wsi_dict[dict_key]["CKD_noDM"] += 1   
#     #             print(filenames[2])
#           except IndexError:
#             pass
#         try:
#             if filenames[1]:
#                 path_sum+=1
#                 print("path 1", filenames[1])
#                 print("prediction: ", class_names[preds[1]])
#                 temp = filenames[1].split("/")[-1].split(".")[0].split('_')[0]
#     #                 dict_key = temp[0]+"_"+temp[1]+"_"+temp[2]+"_"+temp[3]
#                 dict_key = temp
#     #             print(dict_key)                
#                 if dict_key in wsi_all:
#                     if class_names[preds[1]] == "Has_DKD":
#                         wsi_dict[dict_key]["Has_DKD"] += 1
#                     elif class_names[preds[1]] == "Healthy":
#                         wsi_dict[dict_key]["Healthy"] += 1
#                     elif class_names[preds[1]] == "CKD_noDM":
#                         wsi_dict[dict_key]["CKD_noDM"] += 1   
#     #             print(filenames[2])
#           except IndexError:
#             pass
#         try:
#             if filenames[2]:
#                 path_sum+=1
#                 print("path 2", filenames[2])
#                 print("prediction: ", class_names[preds[2]])
#                 temp = filenames[2].split("/")[-1].split(".")[0].split('_')[0]
#                 dict_key = temp
# #                     dict_key = temp[0]+"_"+temp[1]+"_"+temp[2]+"_"+temp[3]
# #                 print(dict_key)                    
#                 if dict_key in wsi_all:
#                     if class_names[preds[2]] == "Has_DKD":
#                         wsi_dict[dict_key]["Has_DKD"] += 1
#                     elif class_names[preds[2]] == "Healthy":
#                         wsi_dict[dict_key]["Healthy"] += 1
#                     elif class_names[preds[2]] == "CKD_noDM":
#                         wsi_dict[dict_key]["CKD_noDM"] += 1   
#         except IndexError:
#             pass

#         try:
#             if filenames[3]:
#                 path_sum+=1
#                 print("path 3", filenames[3])
#                 print("prediction: ", class_names[preds[3]])
#                 temp = filenames[3].split("/")[-1].split(".")[0].split('_')[0]
#                 dict_key = temp
# #                     dict_key = temp[0]+"_"+temp[1]+"_"+temp[2]+"_"+temp[3]
# #                 print(dict_key)
#                 if dict_key in wsi_all:
#                     if class_names[preds[3]] == "Has_DKD":
#                         wsi_dict[dict_key]["Has_DKD"] += 1
#                     elif class_names[preds[3]] == "Healthy":
#                         wsi_dict[dict_key]["Healthy"] += 1
#                     elif class_names[preds[3]] == "CKD_noDM":
#                         wsi_dict[dict_key]["CKD_noDM"] += 1   
#         except IndexError:
#             pass

# # print('Accuracy of the network on the 10000 test images: %d %%' % (
# #     100 * correct / total))


# # prepare to count predictions for each class
# correct_pred = {classname: 0 for classname in class_names}
# total_pred = {classname: 0 for classname in class_names}

# # again no gradients needed
# with torch.no_grad():
#     for i,data in enumerate(dataloader_test):
#         images, labels, paths = data
#         images = images.cuda()
#         labels = labels.cuda()
#         outputs = model_conv(images)
#         _, predictions = torch.max(outputs, 1)
#         # collect the correct predictions for each class
#         for label, prediction in zip(labels, predictions):
#             if label == prediction:
#                 correct_pred[class_names[label]] += 1
#             total_pred[class_names[label]] += 1


# # # print accuracy for each class
# for classname, correct_count in correct_pred.items():
#     accuracy = 100 * float(correct_count) / total_pred[classname]
#     print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
#                                                    accuracy))

visualize_model(model,6,dataloader_test)    
print("Testing & Validation Slide Status: ")
# print(wsi_all)
for k in wsi_all:
#     print(k)
#     print("WSI: ", k, " healthy: ", wsi_dict[k]['Healthy'], "has DKD: ", wsi_dict[k]['Has_DKD'],"CKD_noDM" ,wsi_dict[k]['CKD_noDM'])
    if (max(wsi_dict[k]['Healthy'], wsi_dict[k]['Has_DKD'],wsi_dict[k]['CKD_noDM']) == wsi_dict[k]['Healthy']) and (wsi_dict[k]['Healthy'] != 0 or wsi_dict[k]['Has_DKD'] != 0 or wsi_dict[k]['CKD_noDM'] != 0 ):
        print("WSI: ", k, "Status: Healthy" )
    elif max(wsi_dict[k]['Healthy'], wsi_dict[k]['Has_DKD'],wsi_dict[k]['CKD_noDM']) == wsi_dict[k]['Has_DKD'] and (wsi_dict[k]['Healthy'] != 0 or wsi_dict[k]['Has_DKD'] != 0 or wsi_dict[k]['CKD_noDM'] != 0 ):
        print("WSI: ", k, "Status: Has_DKD" )
    elif max(wsi_dict[k]['Healthy'], wsi_dict[k]['Has_DKD'],wsi_dict[k]['CKD_noDM']) == wsi_dict[k]['CKD_noDM'] and (wsi_dict[k]['Healthy'] != 0 or wsi_dict[k]['Has_DKD'] != 0 or wsi_dict[k]['CKD_noDM'] != 0 ):
        print("WSI: ", k, "Status: CKD_noDM" )

In [5]:
import pandas as pd
from openslide import OpenSlide
df_labels = pd.read_csv("Test_labels.csv")
df_labels

,filenames,labels
0,502d2911-4815-44e0-b66f-b1910e8808e9_S-1910-00...,H-CKD
1,dbdcd9a2-62a7-4988-86a2-90cf1ea57242_S-1910-00...,H-CKD
2,5d07f47b-dd32-4c09-92ea-fd084f4ca2e8_18-162_PA...,HEALTHY
3,829067a5-eda4-415d-8e2e-fa9a15ac583c_18-162_PA...,HEALTHY
4,61a1c11d-d869-4868-ba9b-5f31fbd1dbff_18-162_PA...,HEALTHY
5,7daf471c-957e-4ebf-a206-bc0fef11cf2f_18-162_PA...,HEALTHY
6,760649cb-be20-4559-8f29-5fb4a3225e43_18-162_PA...,HEALTHY
7,16dc8e6f-524c-48cb-9252-46140fd74090_18-162_PA...,HEALTHY
8,2fda166d-a6d0-46a7-acfa-124e0e51dce2_S-2001-01...,DKD
9,060e75ad-0819-4007-b5bc-18a39dbcefdd_S-2001-01...,DKD


In [6]:
from torch.utils.data import Dataset
class MyDaatset(Dataset):
    def __init__(self, path, labels_wsi,transform):
        self.files = glob.glob(path)
        print(type(self.files))
        self.transform = transform
        self.labels = labels_wsi
    def __getitem__(self, item):
        file = self.files[item]
        name = file
        label = self.labels
        file = Image.open(file)
        file = self.transform(file)
        if label.startswith("H-CKD"):
            label=0
        elif label.startswith("Healthy"):
            label=2
        elif label.startswith("Has_DKD"):
            label=1
        return file,label,name
    def __len__(self):
        return len(self.files)


print("hei")

hei


In [25]:
#so i pass a wsi into the function
# inside the function i get the size of the wsi as well as the tsv and patches for it
# so the data loader essentially tests only the values of 1 image at a time.
from PIL import Image, ImageOps, ImageEnhance 
import openslide
import math
def slide_to_scaled_pil_image(slide, SCALE_FACTOR):

    large_w, large_h = slide.dimensions
    new_w = math.floor(large_w / SCALE_FACTOR)
    new_h = math.floor(large_h / SCALE_FACTOR)
    level = slide.get_best_level_for_downsample(SCALE_FACTOR)
    whole_slide_image = slide.read_region((0, 0), level, slide.level_dimensions[level])
    whole_slide_image = whole_slide_image.convert("RGB")
    img = whole_slide_image.resize((new_w, new_h), Image.BILINEAR)
    return img, large_w, large_h, new_w, new_h



def heatmap_generation(model,wsi_name,label):
    SCALE_FACTOR=64
    wsi_path = '/data1/DKDimages/ALL_WSI/'+wsi_name+'.svs'
    print(wsi_path)
    print(os.path.exists(wsi_path))
    wsi_image = OpenSlide(wsi_path)
    # pilImage = Image.open(wsi_path)
    img = openslide.OpenSlide(wsi_path)
    ds_img, large_w, large_h, new_w, new_h = slide_to_scaled_pil_image(img, SCALE_FACTOR)
    ds_img.save("test.png")
    # image = openslide.open_slide(wsi_path)
    # new_IM = Image.convert(image)
    # print(image.dimensions)
    path_sum = 0
    
    print(wsi_image.dimensions)
    print(ds_img.size)
    target_size = ds_img.size
    heat_map = Image.new('RGB', target_size)
    tsv_loc = '/data1/DKDimages/Pyhist_WSI_Image_Coords/' + wsi_name+ '/tile_selection.tsv'
    print(os.path.exists(tsv_loc))
    patch_loc = '/data1/DKDimages/All_patch_Folders/' + wsi_name+'_tiles'
    print(patch_loc)
    print(os.path.exists(patch_loc))
    tsv_coords = pd.read_csv(tsv_loc,sep='\t')
    # print(tsv_coords)
    # print(glob.glob(patch_loc+'/*'))
    temp = MyDaatset('/data1/DKDimages/All_patch_Folders/502d2911-4815-44e0-b66f-b1910e8808e9_S-1910-000043_PAS_1of2_tiles/*',labels_wsi = label,transform=data_transforms)
    # print(len(temp))
    # image_datasets_test_new = tempFolder('/data1/DKDimages/All_patch_Folders/502d2911-4815-44e0-b66f-b1910e8808e9_S-1910-000043_PAS_1of2_tiles',labels = label,transform = data_transforms)
    dataloader_test_new = torch.utils.data.DataLoader(temp, batch_size=4,
                                             shuffle=False, num_workers=8)  
    model.eval()

    with torch.no_grad():
        for i, data in enumerate(dataloader_test_new):
            inputs, labels, filenames = data
            # print(inputs,labels,filenames)
            # print(type(labels))
            inputs = inputs.to(device)
            labels = labels.to(device)
              

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
        
            # print(preds)
            try:
                if filenames[0]: 
                    path_sum+=1
                    # print("path 0", filenames[0])
                    # print("prediction: ", class_names[preds[0]])
                    temp = filenames[0].split("/")[-1].split(".")[0]
    #                 dict_key = temp[0]+"_"+temp[1]+"_"+temp[2]+"_"+temp[3]
                    dict_key = temp
                    # print(dict_key)

                    if class_names[preds[0]] == "Has_DKD":
                        # heat = np.full(shape=(512, 512,3),fill_value=(0,0,255)).astype(np.uint32)
                        # heat = Image.fromarray((heat * 255).astype(np.uint8)) 
                        heat = Image.new('RGB',(int(512/SCALE_FACTOR),int(512/SCALE_FACTOR)),(255,0,0))
                    elif class_names[preds[0]] == "Healthy":
                        # heat = np.full(shape=(512, 512,3),fill_value=(0,255,0)).astype(np.uint32)
                        # heat = Image.fromarray((heat * 255).astype(np.uint8))
                        heat = Image.new('RGB',(int(512/SCALE_FACTOR),int(512/SCALE_FACTOR)),(0,255,0)) 
                    elif class_names[preds[0]] == "H-CKD":
                        # heat = np.full(shape=(512, 512,3),fill_value=(255,0,0)).astype(np.uint32)
                        # heat = Image.fromarray((heat * 255).astype(np.uint8))
                        heat = Image.new('RGB',(int(512/SCALE_FACTOR),int(512/SCALE_FACTOR)),(0,0,255)) 
                    # heat = Image.new('RGB',(512,512),(255,0,0))
                    # heat.save("just_img.png")
                    if (tsv_coords[tsv_coords["Tile"] == (dict_key)]['Keep'].iloc[0] == 1):
                        
                        x = int((tsv_coords[tsv_coords["Tile"] == (dict_key)]['Column'].iloc[0])*int(512)/SCALE_FACTOR)
                        y = int((tsv_coords[tsv_coords["Tile"] == (dict_key)]['Row'].iloc[0])*int(512)/SCALE_FACTOR)
                        print((tsv_coords[tsv_coords["Tile"] == (dict_key)]['Keep'].iloc[0])) 
                        heat_map.paste(im=heat, box=(x,y))

                    # print(x,y)
            except IndexError:
                pass
            try:
                
                if filenames[1]:
                    path_sum+=1
                    # print("path 1", filenames[1])
                    # print("prediction: ", class_names[preds[1]])
                    temp = filenames[1].split("/")[-1].split(".")[0]
                    dict_key = temp
                    # print(dict_key)                

                    if class_names[preds[1]] == "Has_DKD":
                        #  heat = np.full(shape=(512, 512,3),fill_value=(0,0,255)).astype(np.uint32)
                        #  heat = Image.fromarray((heat * 255).astype(np.uint8))
                         heat = Image.new('RGB',(int(512/SCALE_FACTOR),int(512/SCALE_FACTOR)),(255,0,0)) 
                    elif class_names[preds[1]] == "Healthy":
                        #  heat = np.full(shape=(512, 512,3),fill_value=(0,255,0)).astype(np.uint32)
                        #  heat = Image.fromarray((heat * 255).astype(np.uint8))
                         heat = Image.new('RGB',(int(512/SCALE_FACTOR),int(512/SCALE_FACTOR)),(0,255,0)) 
                    elif class_names[preds[1]] == "H-CKD":
                        #  heat = np.full(shape=(512, 512,3),fill_value=(255,0,0)).astype(np.uint32)
                        #  heat = Image.fromarray((heat * 255).astype(np.uint8))
                         heat = Image.new('RGB',(int(512/SCALE_FACTOR),int(512/SCALE_FACTOR)),(0,0,255)) 
                    
                    if (tsv_coords[tsv_coords["Tile"] == (dict_key)]['Keep'].iloc[0] == 1):
                        
                        x = int((tsv_coords[tsv_coords["Tile"] == (dict_key)]['Column'].iloc[0])*int(512)/SCALE_FACTOR)
                        y = int((tsv_coords[tsv_coords["Tile"] == (dict_key)]['Row'].iloc[0])*int(512)/SCALE_FACTOR)
                        print((tsv_coords[tsv_coords["Tile"] == (dict_key)]['Keep'].iloc[0])) 
                        heat_map.paste(im=heat, box=(x,y))
                    
    #             print(filenames[2])
            except IndexError:
                pass
            try:
                if filenames[2]:
                    path_sum+=1
                    # print("path 2", filenames[2])
                    # print("prediction: ", class_names[preds[2]])
                    temp = filenames[2].split("/")[-1].split(".")[0]
                    dict_key = temp
#                     dict_key = temp[0]+"_"+temp[1]+"_"+temp[2]+"_"+temp[3]
                    # print(dict_key)                    

                    if class_names[preds[2]] == "Has_DKD":
                        # heat = np.full(shape=(512, 512,3),fill_value=(0,0,255)).astype(np.uint32)
                        # heat = Image.fromarray((heat * 255).astype(np.uint8))
                        heat = Image.new('RGB',(int(512/SCALE_FACTOR),int(512/SCALE_FACTOR)),(255,0,0)) 
                    elif class_names[preds[2]] == "Healthy":
                        # heat = np.full(shape=(512, 512,3),fill_value=(0,255,0)).astype(np.uint32)
                        # heat = Image.fromarray((heat * 255).astype(np.uint8))
                        heat = Image.new('RGB',(int(512/SCALE_FACTOR),int(512/SCALE_FACTOR)),(0,255,0)) 
                    elif class_names[preds[2]] == "H-CKD":
                        # heat = np.full(shape=(512, 512,3),fill_value=(255,0,0)).astype(np.uint32)
                        # heat = Image.fromarray((heat * 255).astype(np.uint8))
                        heat = Image.new('RGB',(int(512/SCALE_FACTOR),int(512/SCALE_FACTOR)),(0,0,255)) 
                    
                    if (tsv_coords[tsv_coords["Tile"] == (dict_key)]['Keep'].iloc[0] == 1):
                        
                        x = int((tsv_coords[tsv_coords["Tile"] == (dict_key)]['Column'].iloc[0])*int(512)/SCALE_FACTOR)
                        y = int((tsv_coords[tsv_coords["Tile"] == (dict_key)]['Row'].iloc[0])*int(512)/SCALE_FACTOR)
                        print((tsv_coords[tsv_coords["Tile"] == (dict_key)]['Keep'].iloc[0])) 
                        heat_map.paste(im=heat, box=(x,y))  
            except IndexError:
                pass

            try:
                if filenames[3]:
                    path_sum+=1
                    # print("path 3", filenames[3])
                    # print("prediction: ", class_names[preds[3]])
                    temp = filenames[3].split("/")[-1].split(".")[0]
                    dict_key = temp
#                     dict_key = temp[0]+"_"+temp[1]+"_"+temp[2]+"_"+temp[3]
                    # print(dict_key)

                    if class_names[preds[3]] == "Has_DKD":
                        # heat = np.full(shape=(512, 512, 3),fill_value=(0,0,255)).astype(np.uint32)
                        # heat = Image.fromarray((heat * 255).astype(np.uint8))
                        heat = Image.new('RGB',(int(512/SCALE_FACTOR),int(512/SCALE_FACTOR)),(255,0,0))
                    elif class_names[preds[3]] == "Healthy":
                        # heat = np.full(shape=(512, 512, 3),fill_value=(0,255,0)).astype(np.uint32)
                        # heat = Image.fromarray((heat * 255).astype(np.uint8))
                        heat = Image.new('RGB',(int(512/SCALE_FACTOR),int(512/SCALE_FACTOR)),(0,255,0))
                    elif class_names[preds[3]] == "H-CKD":
                        # heat = np.full(shape=(512, 512, 3),fill_value=(255,0,0)).astype(np.uint32)
                        # heat = Image.fromarray((heat * 255).astype(np.uint8))
                        heat = Image.new('RGB',(int(512/SCALE_FACTOR),int(512/SCALE_FACTOR)),(0,0,255))
                    
                    if (tsv_coords[tsv_coords["Tile"] == (dict_key)]['Keep'].iloc[0] == 1):
                        
                        x = int((tsv_coords[tsv_coords["Tile"] == (dict_key)]['Column'].iloc[0])*int(512)/SCALE_FACTOR)
                        y = int((tsv_coords[tsv_coords["Tile"] == (dict_key)]['Row'].iloc[0])*int(512)/SCALE_FACTOR)
                        print((tsv_coords[tsv_coords["Tile"] == (dict_key)]['Keep'].iloc[0])) 
                        heat_map.paste(im=heat, box=(x,y))
                    
            except IndexError:
                pass
        heat_map.save('mask.png')
        downsampled_img = Image.blend(ds_img, heat_map, 0.4)
        downsampled_img.save("heatmap.png")
        print(path_sum)
        
        
test_list = df_labels["filenames"].tolist()
print(test_list[0])
wsi_label = df_labels[df_labels["filenames"] == (test_list[0])]['labels'].iloc[0]
print(wsi_label)
model = torch.load(PATH)
model.eval()
heatmap_generation(model,test_list[0],wsi_label) 

502d2911-4815-44e0-b66f-b1910e8808e9_S-1910-000043_PAS_1of2
H-CKD
/data1/DKDimages/ALL_WSI/502d2911-4815-44e0-b66f-b1910e8808e9_S-1910-000043_PAS_1of2.svs
True
(139509, 53875)
(2179, 841)
True
/data1/DKDimages/All_patch_Folders/502d2911-4815-44e0-b66f-b1910e8808e9_S-1910-000043_PAS_1of2_tiles
True
<class 'list'>


TypeError: integer argument expected, got float